In [2]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM,TrainingArguments,Trainer,DataCollatorForLanguageModeling,TextDataset
from peft import LoraConfig,TaskType,get_peft_model
from datasets import load_dataset
import json

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
path = 'D:\Programming\Storage\Class\Project_F\output_dir'
model = AutoModelForCausalLM.from_pretrained(path)

OSError: D:\Programming\Storage\Class\Project_F\output_dir does not appear to have a file named config.json. Checkout 'https://huggingface.co/D:\Programming\Storage\Class\Project_F\output_dir/None' for available files.

In [1]:
import torch
from peft import PeftModel,LoraConfig,TaskType
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = 'openai-community/gpt2'
adapters_name = "output_dir"


In [ ]:

print(f"Starting to load the model {model_name} into memory")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    #torch_dtype=torch.bfloat16,
    #device_map={"": 0}
)
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r = 1,
    lora_alpha=32,
    lora_dropout=0.1
)
model = PeftModel(model,peft_config=peft_config)
model.print_trainable_parameters()
m = PeftModel.from_pretrained(model, adapters_name,local_files_only = True)
#m = m.merge_and_unload()
m.print_trainable_parameters()
tok = AutoTokenizer.from_pretrained(model_name)
tok.bos_token_id = 1

stop_token_ids = [0]

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model openai-community/gpt2 into memory


c:\Users\Anh\AppData\Local\Programs\Python\Python310\lib\site-packages\peft\tuners\lora\layer.py:861: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 36,864 || all params: 124,476,672 || trainable%: 0.029615187655402612
trainable params: 0 || all params: 125,619,456 || trainable%: 0.0
Successfully loaded the model openai-community/gpt2 into memory


In [3]:
import time

In [ ]:
def generate_text(model, input_text,max_length = 256):
    start = time.time()
    output = model.base_model().generate(input_text,max_length=max_length)
    print('\n Ouput : \n',output)
    end = time.time()
    print(f'Total time elapsed : {end-start:.2f}s')

In [5]:
model_outputs = model.generate(**inputs,max_new_tokens = 32,return_dict_in_generate= True,output_scores=True)
generated_tokens_ids = model_outputs.sequences[0]

tokenizer.decode(generated_tokens_ids)


NameError: name 'model' is not defined

In [7]:
from transformers import pipeline
generator = pipeline('text-generation',model=model_name)
generator('Who are you ?')

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
c:\Users\Anh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Who are you ? Can you believe all these crazy guys I saw here?"\n\nTyson, who had been attending the event with a friend, walked up to him. "You two have some crazy issues," he said, "can you believe'}]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
x = "Who are you ?"
inputs = tokenizer(x, return_tensors="pt")
model = AutoModelForCausalLM.from_pretrained(
    'model',
    #load_in_4bit=True,
    #torch_dtype=torch.bfloat16,
    #device_map={"": 0}
)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r = 1,
    lora_alpha=32,
    lora_dropout=0.1
)
perft_config = {
    'max_tokens': 1024
}
model = PeftModel(model,peft_config=peft_config)
#model.load_adapter('output_dir',adapters_name)
#model.set_adapter(adapters_name)
#model.unload()
#model.merge_adapter()
model.print_trainable_parameters()
model_outputs = model.generate(**inputs,max_new_tokens = 32)
generated_tokens_ids = model_outputs.numpy()[0]
print(generated_tokens_ids)
result = tokenizer.decode(generated_tokens_ids)
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


trainable params: 36,864 || all params: 124,476,672 || trainable%: 0.029615187655402612
[8241  389  345 5633  198  198   40  716  257  582  286 1793   13  198
  198   40  716  257  582  286 1793   13  198  198   40  716  257  582
  286 1793   13  198  198   40  716  257]
Who are you?

I am a man of God.

I am a man of God.

I am a man of God.

I am a


In [3]:
print (result)

Good morning.
Assistant: Hello there. Glad to see you're back. How are you feeling today?
Assistant: Hello there. Glad to see you're back


In [69]:
print (result)

Good morning, my dear, my dear, my dear, my dear, my dear, my dear, my dear, my dear, my dear, my dear, my


In [ ]:
[15496    11   314  1101  7926    11   475   314  1101   407  1654   611
   345   821  3910   286   428    13   314  1101   407  1654   611   345
   821  3910   286   428    13   314  1101   407  1654]

In [24]:
for layer in model.named_modules():
    print(layer)

('', GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
))
('transformer', GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(

In [6]:
model_outputs = m.generate(**inputs,max_new_tokens = 32)
generated_tokens_ids = model_outputs.numpy()[0]
print(generated_tokens_ids)
tokenizer.decode(generated_tokens_ids)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[8241  389  345 5633  198  198   40  716  257  582  286 1793   13  198
  198   40  716  257  582  286 1793   13  198  198   40  716  257  582
  286 1793   13  198  198   40  716  257]


'Who are you?\n\nI am a man of God.\n\nI am a man of God.\n\nI am a man of God.\n\nI am a'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id


x = "Hello world, ..."
inputs = tokenizer(x, return_tensors="pt")

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer.pad_token_id = tokenizer.eos_token_id


x = "Hello "
inputs = tokenizer(x, return_tensors="pt")

model_outputs = model.generate(**inputs, max_new_tokens=5, return_dict_in_generate=True, output_scores=True)

generated_tokens_ids = model_outputs.sequences[0]

tokenizer.decode(generated_tokens_ids)

In [ ]:
m.base_model()

In [ ]:
generate_text(m,tokenizer.encode('Hello'))